# TEXT MINING for PRACTICE
- 본 자료는 텍스트 마이닝을 활용한 연구 및 강의를 위한 목적으로 제작되었습니다.
- 본 자료를 강의 목적으로 활용하고자 하시는 경우 꼭 아래 메일주소로 연락주세요.
- 본 자료에 대한 허가되지 않은 배포를 금지합니다.
- 강의, 저작권, 출판, 특허, 공동저자에 관련해서는 문의 바랍니다.
- **Contact : 전병진(fingeredman@gmail.com)**

---

In [1]:
!ls

mecab-python-0.996  sample_data


In [2]:
# TEANAPS (https://github.com/fingeredman/teanaps)
!git clone https://github.com/fingeredman/teanaps.git

Cloning into 'teanaps'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 208 (delta 3), reused 10 (delta 2), pack-reused 193
Receiving objects: 100% (208/208), 27.56 MiB | 21.45 MiB/s, done.
Resolving deltas: 100% (99/99), done.


In [3]:
!ls

mecab-python-0.996  sample_data  teanaps


## WEEK 11-1. 뉴스기사를 활용한 KOSPI 등락여부 예측하기
- Python으로 비정형 데이터를 활용해 KOSPI 등락여부를 예측하는 방법에 대해 다룹니다.

---

In [4]:
from teanaps import configure as con
PLOTLY_USERNAME = con.PLOTLY_USERNAME
PLOTLY_API_KEY = con.PLOTLY_API_KEY

import plotly 
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff
from plotly import tools
plotly.tools.set_credentials_file(username=PLOTLY_USERNAME, api_key=PLOTLY_API_KEY)
plotly.tools.set_config_file(world_readable=False, sharing='private')
init_notebook_mode(connected=True)

from teanaps.nlp.SyntaxAnalyzer import SyntaxAnalyzer
from teanaps.text_analysis.TfidfCalculator import TfidfCalculator
from teanaps.text_analysis.TopicClustering import TopicClustering

#import statsmodels.api as sm
#from statsmodels.tsa.api import VAR, DynamicVAR
from statsmodels.tsa.arima_model import ARIMA
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

import pandas as pd
import numpy as np
import glob

In [5]:
df = pd.read_csv('teanaps/data/kospi.txt', sep="\t") 
df["y"] = [float(y.replace(",", "")) for y in df["y"]]
df

,ds,y,오픈,고가,저가,거래량,변동폭
0,2019년 5월,2061.25,"2,192.94","2,217.03","2,051.69",-,-6.46%
1,2019년 4월,2203.59,"2,153.31","2,252.05","2,153.31",9.69B,2.94%
2,2019년 3월,2140.67,"2,210.97","2,216.02","2,120.93",6.46B,-2.49%
3,2019년 2월,2195.44,"2,211.93","2,241.76","2,167.36",6.88B,-0.43%
4,2019년 1월,2204.85,"2,050.55","2,222.88","1,984.53",9.46B,8.03%
5,2018년 12월,2041.04,"2,127.78","2,136.64","2,014.28",7.63B,-2.66%
6,2018년 11월,2096.86,"2,035.11","2,136.74","2,023.43",6.98B,3.31%
7,2018년 10월,2029.69,"2,349.64","2,352.11","1,985.95",7.02B,-13.37%
8,2018년 9월,2343.07,"2,317.93","2,356.62","2,264.90",5.43B,0.87%
9,2018년 8월,2322.88,"2,301.17","2,322.88","2,218.09",6.20B,1.20%


In [0]:
sa = SyntaxAnalyzer()

document_list = []
tokenized_sentence_list = []

POS_LIST = ["NNG", "NNP"]

num_monthly_documents = 100

file_list = glob.glob("teanaps/data/monthly_data/*")
file_list.sort()
for filename in file_list:
    count = 1
    line_list = open(filename, encoding="utf-8").read().split("\n")
    for line in line_list:
        if count > num_monthly_documents:
            continue
        if len(line) > 100:
            count += 1
            col = line.split("\t")
            datetime = col[0]
            month = datetime[:7]
            content = col[1]
            document_list.append(content)
            tagged_word_list = sa.parse(content)
            tokenized_sentence = ""
            for word, pos in tagged_word_list:
                if pos in POS_LIST:
                    tokenized_sentence += " " + word
            tokenized_sentence_list.append(tokenized_sentence.strip())

In [7]:
tokenized_sentence_list[:5]

['글쎄요 가스 아마 석탄 사용 금지 때문 가격 급등 것 관련 상장 주식 대부분 연상 주춤 경향',
 '중국 거주 얼마 전 중국 주식시장 투자 갓 회사 대한 투자 대해 생각 글 전 가스 얼마 후 매입 갑자 계속 장정 혹시 일전 프로 수익 매도 그 이후 다시 한지 얼마 화 능수 전 투자 혹시 저 한지 얼 회사 주로 투자 분 중국 주식 모두 투하 길',
 '언어 매너 활동 비트 맨 먼저 새해 복 저 주위 가상 화폐 투자 백 기본 천 심지어 삼사억 이득 지인 전 윌 일천만원 투자 결과 현재 로 상 오늘 뉴스 정말 한해 코스닥 상승 열풍 주식시장 폐장 코스닥 상장 시가총액 실체 리플 이나 더 리움 등 가상 화폐 시가총액 이건 대한민국 미래 기업 전체 시가총액 가치 정말 것 고수 고견 한해 비트 맨 정보 공유 비트 맨 무료 리딩 비 매너 회원 신고 투자 본인 책임 정보 판단 비 매너 반말 비속어 공격성 어휘 등 단 톡 방 타 카 페 홍보 시 경고 영구 강퇴 비트코인 더 비트 맨 동호회 비트코인 더 리움 채굴 라이트코인 리플 가상 화폐 투자 정보 비트코인 시세 퀀텀 오스 비트코인 전망 비트 캐쉬 캐시 대쉬 클래식 빗썸 업 비트 코인 코인 네스트',
 '천만원 존버 기회 개인 욕심 급등 코인 다가 손절 반복 네 욕심 한해 복 투하 쇼',
 '투자 코인 지금 자꾸 팔고 사고 방법 좀 보 걱정 스타일 계속 폰 보지']

In [0]:
tfidf = TfidfCalculator()

df_tfidf_monthly = []
for i in range(11):
    tokenized_sentence_list_monthly = tokenized_sentence_list[i*100:(i+1)*100]
    #tokenized_sentence_list_monthly
    tfidf.calculation_tfidf(tokenized_sentence_list_monthly)
    df_tfidf = tfidf.get_tfidf_matrix()
    #df_tfidf
    df_tfidf_monthly.append(df_tfidf)

In [9]:
df_tfidf_monthly[10][:20]

com  company  core  coredictionary  dictionary  fod  how  koreans  market  \
0   0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
1   0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
2   0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
3   0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
4   0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
5   0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
6   0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
7   0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
8   0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
9   0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
10  0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
11  0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
12  0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
13  0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
14  0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
15  0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
16  0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
17  0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
18  0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
19  0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   

    rice  share  staple  stock  today  yons        가감        가격  가격표   가기  \
0    0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
1    0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.175952  0.0  0.0   
2    0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.046736  0.0  0.0   
3    0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.033230  0.0  0.0   
4    0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
5    0.0    0.0     0.0    0.0    0.0   0.0  0.055756  0.000000  0.0  0.0   
6    0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
7    0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
8    0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
9    0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
10   0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
11   0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
12   0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
13   0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
14   0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
15   0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
16   0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
17   0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
18   0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
19   0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   

         가까이   가끔       가능       가능성      가닌까   가담        가도  가뜩이나   가량  가려진  \
0   0.000000  0.0  0.00000  0.000000  0.00000  0.0  0.000000   0.0  0.0  0.0   
1   0.000000  0.0  0.00000  0.014442  0.00000  0.0  0.000000   0.0  0.0  0.0   
2   0.075417  0.0  0.00000  0.000000  0.00000  0.0  0.000000   0.0  0.0  0.0   
3   0.000000  0.0  0.00000  0.000000  0.00000  0.0  0.000000   0.0  0.0  0.0   
4   0.000000  0.0  0.00000  0.000000  0.00000  0.0  0.000000   0.0  0.0  0.0   
5   0.000000  0.0  0.00000  0.000000  0.00000  0.0  0.000000   0.0  0.0  0.0   
6   0.000000  0.0  0.00000  0.000000  0.00000  0.0  0.070839   0.0  0.0  0.0   
7   0.000000  0.0  0.00000  0.000000  0.0000

In [10]:
tc = TopicClustering()

NUM_TOPICS = 20
NUM_KEYWORDS = 20

lda_result = tc.lda_topic_modeling(tokenized_sentence_list, NUM_TOPICS, NUM_KEYWORDS)
lda_result

[(0,
  [('것', 0.030829735),
   ('그', 0.0144633055),
   ('수', 0.012667565),
   ('사람', 0.0101372395),
   ('저', 0.009679858),
   ('주식', 0.007984952),
   ('매수', 0.0073042065),
   ('매도', 0.0069502667),
   ('더', 0.0068364935),
   ('시장', 0.006681913),
   ('말', 0.0059494716),
   ('돈', 0.005241592),
   ('주택', 0.0048917816),
   ('생각', 0.004852598),
   ('제', 0.0047834134),
   ('정도', 0.00455114),
   ('대한', 0.0044929083),
   ('가격', 0.004372357),
   ('때', 0.0043324656),
   ('내', 0.0041484917)]),
 (1,
  [('것', 0.017261146),
   ('주가', 0.016869696),
   ('매', 0.012553838),
   ('중국', 0.012371244),
   ('시장', 0.010807932),
   ('금융', 0.010796339),
   ('주식', 0.01055562),
   ('주식시장', 0.009513217),
   ('수익', 0.007199314),
   ('세계', 0.006745615),
   ('확인', 0.00567935),
   ('상승', 0.0056681456),
   ('변화', 0.0055649006),
   ('바로', 0.0049569756),
   ('전쟁', 0.004931879),
   ('사실', 0.004837914),
   ('공부', 0.0047323937),
   ('추세', 0.004654166),
   ('투자', 0.00450561),
   ('경제', 0.0044791433)]),
 (2,
  [('금리', 0.0167306

In [0]:
hot_keyword_list = []

for num_topics, keyword_list in lda_result:
    for keyword, score in keyword_list:
        if keyword not in hot_keyword_list:
            hot_keyword_list.append(keyword)
            
hot_keyword_dict = {}

for hot_keyword in hot_keyword_list:
    tfidf_series = []
    for df_tfidf in df_tfidf_monthly:
        tfidf_dict = dict(df_tfidf.sum())
        if hot_keyword in tfidf_dict.keys():
            tfidf_series.append(tfidf_dict[hot_keyword])
        else:
            tfidf_series.append(0)
    tfidf_series2 = []
    for idx in range(len(tfidf_series)-1):
        if tfidf_series[idx] > tfidf_series[idx+1]:
            tfidf_series2.append(0)
        else:
            tfidf_series2.append(1)
    hot_keyword_dict[hot_keyword] = tfidf_series2

In [12]:
d = df[["y"]].values[6:17]
d = list(d)
d = [float(dd) for dd in d]
d

[2096.86,
 2029.69,
 2343.07,
 2322.88,
 2295.26,
 2326.13,
 2423.01,
 2515.38,
 2445.85,
 2427.36,
 2566.46]

In [13]:
tc.set_plotly()

x = [i for i in range(11)]

data = [go.Scatter(
        x=x,
        y=d,
        name="KOSPI",
        marker=dict(
            #color='blue',
        ),
        yaxis='y2'
    )]


for key, value in hot_keyword_dict.items():
    y = value
    data.append(go.Scatter(
        x=x,
        y=y,
        name=key,
        marker=dict(
            #color='blue',
        ),
        yaxis='y'
    ),)
layout = go.Layout(
    title='Sequence LDA Model Topic Trend',
    xaxis=dict(
        title='TIME SLICE',
        titlefont=dict(
            size=10,
            color='black'
        ),
        dtick = 1,
        showticklabels=True,
        tickangle=0,
        tickfont=dict(
            size=10,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='WEIGHT',
        titlefont=dict(
            size=10,
            color='black'
        ),
        showticklabels=True,
        tickangle=0,
        tickfont=dict(
            size=10,
            color='black'
        ),
        exponentformat='e',
        showexponent='all',
        #overlaying='y',
    ),
    yaxis2=dict(
                title='TF-IDF',
                titlefont=dict(
                    size=10,
                    color='black'
                ),
                showticklabels=True,
                tickangle=0,
                tickfont=dict(
                    size=10,
                    color='black'
                ),
                overlaying='y',
                side='right'
            ),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='TF-IDF Graph')

In [14]:
y = df["y"].values[6:17]
ud_y = []

for y_idx in range(len(y)-1):
    print(y[y_idx], y[y_idx+1])
    if y[y_idx] < y[y_idx+1]:
        ud_y.append(1)
    else:
        ud_y.append(0)

2096.86 2029.69
2029.69 2343.07
2343.07 2322.88
2322.88 2295.26
2295.26 2326.13
2326.13 2423.01
2423.01 2515.38
2515.38 2445.85
2445.85 2427.36
2427.36 2566.46


In [15]:
Y = pd.DataFrame(ud_y)
Y = Y[0].values

X = pd.DataFrame(hot_keyword_dict)
X = X.values[:]

df = pd.DataFrame(X, columns = hot_keyword_dict.keys())
df['target'] = Y
df

,것,그,수,사람,저,주식,매수,매도,더,시장,말,돈,주택,생각,제,정도,대한,가격,때,내,주가,매,중국,금융,주식시장,수익,세계,확인,상승,변화,바로,전쟁,사실,공부,추세,투자,경제,금리,미국,뉴스,무역,하락,달러,인상,증시,연합뉴스,지수,마감,관세,중,종목,대출,저축은행,차트,주,추천,지금,글,오늘,날,코인,비트코인,비트,화폐,정보,가상,맨,거래,리플,암호,매너,등,블록,체인,평가,수익률,금액,국내,증가,이유,국민연금,듯,매입,실적,홀딩스,목표,서비스,우리,펀드,베트남,정부,비,해외,울산,폭락,때문,기업,개인,부동산,책,이후,강의,공매도,분,요,배당,증권,한국,금융투자,대비,삼성,하나,전일,전망,전자,산업,재무,대통령,보험,설계,관리,축구,중계,위,문제,무료,김희선,에이즈,자산,상품,부자,방법,여러분,카페,비상,가입,손실,분석,관련,아파트,혁신도시,직원,기관,지역,분양,수요,공급,주변,차,집,수도권,특별,니스,바이오,강남,서울,신부,직스,의견,보통주,자신,장외,기법,단계,고수,좀,곳,진짜,시간,가지,단,금,학점,투기,거,알,대해,시험,상황,이란,성장,률,품질,트럼프,핵,회사,이번,target
0,1,1,1,1,1,1,1,0,1,0,1,1,1,0,1,1,0,1,1,1,1,1,0,0,1,1,0,1,1,0,1,0,0,1,1,0,0,1,0,1,0,1,0,0,0,1,0,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,0,1,0,1,1,0,0,1,1,1,1,0,1,0,0,0,1,0,1,0,0,0,1,1,1,0,1,1,1,0,1,1,0,0,1,1,1,0,1,0,1,1,0,0,0,0,0,1,0,1,0,1,0,1,1,0,1,1,1,1,1,0,1,1,1,0,0,1,0,1,1,0,1,0,0,0,1,1,0,1,1,1,1,1,0,1,1,1,1,0,1,1,1,0,1,1,1,0,1,0,0,0,1,1,0,1,1,1,1,1,0,1,1,0,1,1,1,0,1,1,1,1,0,0,0
1,1,1,1,1,1,0,0,0,1,0,1,1,0,1,1,0,1,0,1,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,1,0,0,1,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,1,1,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0,1,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,1,0,1,1,0,1,0,0,0,1,1,0,1,0,1,1
2,1,1,1,0,1,1,1,0,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,0,0,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0
3,1,1,1,1,1,0,1,1,1,0,1,1,0,1,1,1,0,0,1,1,0,1,0,0,1,1,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,1,1,1,0,0,0,0,0,1,0,1,1,0,1,0,0,0,1,1,0,0,0,1,1,1,0,0,0,0,0,1,1,0,1,1,1,1,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,1,1,0,0,1,1,0,0,1,0,0,1,0,0,0,0,1,1,1,1,1,0,0,0,0,1,1,1,1,0,1,1,0,0,1,1,1,0,1,0,0,1,1,1,1,1,0,0,1,1,1,0,1,1,1,1,0,1,0,1,0,1,1,1,1,0
4,1,1,1,1,1,0,0,0,1,0,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,1,1,0,1,1,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,1,0,1,1,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,1,1,1,1,0,1,1,0,0,0,0,0,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,0,1,0,0,1,0,0,0,1,1,1,0,1,1,1,1,1,1,1,0,1,0,0,0,1,1,0,1,1,0,1
5,1,1,1,0,1,0,1,1,1,1,1,1,0,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,1,1,1,0,1,1,1,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,1,1,1,1,0,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1
6,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,0,1,1,1,1,0,1,1,1,1,1,0,1,1,1,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,0,1,0,1,1,1,1,1,1,0,1,0,0,0,1,1,1,1,1,1,0,0,0,1,0,0,0,0,1,1,1,0,1,0,0,1,1,1,1,1,1,1,0,1,1,1,0,0,0,1,0,0,0,0,0,0,1,0,1,1,1,1,1,0,1,1,0,1,0,0,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,0,1,0,1,1,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,0,0,1
7,1,1,1,0,1,0,0,0,1,0,1,1,0,1,1,0,0,1,1,1,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,1,1,1,1,1,1,1,0,0,1,0,1,1,0,0,1,0,0,0,1,1,0,0,1,1,1,1,1,1,1,0,1,1,0,0,0,0,0,1,1,0,0,1,1,1,1,0,1,1,0,0,1,0,1,0,0,1,0,0,1,1,1,0,0,1,1,0,1,1,1,1,0,0,1,0,1,1,0,1,1,1,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,0,1,1,0,1,1,0,0
8,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,0,0,1,1,1,1,1,1,0,1,0,0,1,1,1,0,1,0,1,1,0,0,0,1,0,0,1,0,1,1,1,0,1,1,1,0,1,1,1,0,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,0,1,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,1,1,0,0,1,1,0,1,1,0,0,1,1,0,1,0,1,1,0,1,1,1,1,1,0,1,0,1,0,1,1,0,1,1,1,0,1,1,0,0,0,1,1,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,0,1,1,1,1,0,1,0,1,0
9,1,1,1,0,1,1,1,0,1,1,1,1,0,1,1,1

In [16]:
for i in range(5):
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=None)
    dtc = DecisionTreeClassifier(random_state=0)
    #dtc = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)
    dtc.fit(x_train, y_train)
    score = dtc.score(x_test, y_test)
    print('Accuracy', i+1, ' :', score)

Accuracy 1  : 0.6666666666666666
Accuracy 2  : 0.6666666666666666
Accuracy 3  : 0.0
Accuracy 4  : 0.6666666666666666
Accuracy 5  : 1.0


In [18]:
tc.set_plotly()

x = [i for i in range(11)]

data = [go.Scatter(
        x=x,
        y=d,
        name="KOSPI",
        marker=dict(
            #color='blue',
        ),
        yaxis='y2'
    ),
    go.Scatter(
        x=x,
        y=ud_y,
        name="Label",
        marker=dict(
            #color='blue',
        ),
        yaxis='y'
    ),
    go.Scatter(
        x=x,
        y=dtc.predict(X),
        name="Predict",
        marker=dict(
            #color='blue',
        ),
        yaxis='y'
    )]

layout = go.Layout(
    title='Predictive Analysis',
    xaxis=dict(
        title='TIME SLICE',
        titlefont=dict(
            size=10,
            color='black'
        ),
        dtick = 1,
        showticklabels=True,
        tickangle=0,
        tickfont=dict(
            size=10,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='DOWN <-------------------------------------------> UP',
        titlefont=dict(
            size=10,
            color='black'
        ),
        showticklabels=True,
        tickangle=0,
        tickfont=dict(
            size=10,
            color='black'
        ),
        dtick = 1,
        exponentformat='e',
        showexponent='all',
        #overlaying='y',
    ),
    yaxis2=dict(
                title='KOSPI200',
                titlefont=dict(
                    size=10,
                    color='black'
                ),
                showticklabels=True,
                tickangle=0,
                tickfont=dict(
                    size=10,
                    color='black'
                ),
                overlaying='y',
                side='right'
            ),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='TF-IDF Graph')